In [47]:
import pandas as pd
from typing import List, Dict, Callable, Optional
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import MinMaxScaler


# La función

In [ ]:
def preparar_tabla_secundaria(
    df: pd.DataFrame,
    id_nivel_1: str,
    id_cliente: str = "SK_ID_CURR",
    cols_drop_logicas: Optional[List[str]] = None,
    transformaciones: Optional[Dict[str, Callable[[pd.Series], pd.Series]]] = None,
    agregaciones_nivel_1: Optional[Dict[str, List[str]]] = None,
    agregaciones_cliente: Optional[Dict[str, List[str]]] = None,
    prefijo: str = ""
) -> pd.DataFrame:
    """
    Prepara y agrega una tabla secundaria siguiendo un pipeline estandarizado.
    Soporta tablas con y sin SK_ID_CURR.
    """

    df_proc = df.copy()

    # ---------------------------
    # 1. Validación mínima
    # ---------------------------
    if id_nivel_1 not in df_proc.columns:
        raise ValueError(f"La columna obligatoria '{id_nivel_1}' no existe en el DataFrame.")

    # ---------------------------
    # 2. Eliminación de columnas por lógica estructural
    # ---------------------------
    if cols_drop_logicas:
        df_proc.drop(columns=cols_drop_logicas, errors="ignore", inplace=True)

    # ---------------------------
    # 3. Eliminación de columnas constantes
    # ---------------------------
    constant_cols = [c for c in df_proc.columns if df_proc[c].nunique(dropna=False) <= 1]
    df_proc.drop(columns=constant_cols, inplace=True)

    # ---------------------------
    # 4. Transformaciones simples
    # ---------------------------
    if transformaciones:
        for col, func in transformaciones.items():
            if col in df_proc.columns:
                df_proc[col] = func(df_proc[col])

    # ---------------------------
    # 5. Agregación al nivel natural
    # ---------------------------
    if not agregaciones_nivel_1:
        raise ValueError("Se deben definir agregaciones para el nivel natural.")

    nivel_1 = (
        df_proc
        .groupby(id_nivel_1)
        .agg(agregaciones_nivel_1)
    )

    nivel_1.columns = [
        f"{prefijo}{c}_{agg}" for c, agg in nivel_1.columns
    ]
    nivel_1.reset_index(inplace=True)

    # ---------------------------
    # 6. Agregación a nivel cliente (opcional)
    # ---------------------------
    if agregaciones_cliente and id_cliente in df_proc.columns:

        mapa_cliente = df_proc[[id_nivel_1, id_cliente]].drop_duplicates()

        nivel_cliente = (
            nivel_1
            .merge(mapa_cliente, on=id_nivel_1, how="left")
            .groupby(id_cliente)
            .agg(agregaciones_cliente)
        )

        nivel_cliente.columns = [
            f"{prefijo}{c}_{agg}" for c, agg in nivel_cliente.columns
        ]
        nivel_cliente.reset_index(inplace=True)

        return nivel_cliente

    # ---------------------------
    # 7. Retorno por nivel natural
    # ---------------------------
    return nivel_1


# Rutas

In [ ]:
DATA_PATH = "../datos_examen"

application = pd.read_parquet(os.path.join(DATA_PATH, "application_train.parquet"))
bureau = pd.read_parquet(os.path.join(DATA_PATH, "bureau.parquet"))
bureau_balance = pd.read_parquet(os.path.join(DATA_PATH, "bureau_balance.parquet"))
previous_application = pd.read_parquet(os.path.join(DATA_PATH, "previous_application.parquet"))
pos_cash = pd.read_parquet(os.path.join(DATA_PATH, "POS_CASH_balance.parquet"))
installments = pd.read_parquet(os.path.join(DATA_PATH, "installments_payments.parquet"))
credit_card = pd.read_parquet(os.path.join(DATA_PATH, "credit_card_balance.parquet"))
columns_desc = pd.read_parquet(os.path.join(DATA_PATH, "HomeCredit_columns_description.parquet"))

In [ ]:
data_set_base = application.copy()

# Preparación de bureau_balance

In [ ]:
# ============================================================
# 1. TRANSFORMACIÓN DE bureau_balance (nivel mensual → crédito)
# ============================================================

features_bureau_balance = preparar_tabla_secundaria(
    df=bureau_balance,
    id_nivel_1="SK_ID_BUREAU",
    cols_drop_logicas=["MONTHS_BALANCE"],   # no aporta señal al resumir
    transformaciones={
        # STATUS representa meses de atraso
        "STATUS": lambda s: s.map({
            "0": 0, "1": 1, "2": 2, "3": 3, "4": 4, "5": 5,
            "C": 0,   # crédito cerrado, sin atraso
            "X": 0    # sin información, se asume sin atraso
        })
    },
    agregaciones_nivel_1={
        "STATUS": ["mean", "max"]
    },
    prefijo="bb_"
)

# Renombrar columnas a nombres semánticos
features_bureau_balance = features_bureau_balance.rename(columns={
    "bb_STATUS_mean": "bb_avg_months_past_due",
    "bb_STATUS_max": "bb_max_months_past_due"
})

# ============================================================
# 2. ASOCIAR CADA CRÉDITO CON SU CLIENTE (JOIN con bureau)
# ============================================================

bb_with_client = features_bureau_balance.merge(
    bureau[["SK_ID_BUREAU", "SK_ID_CURR"]],
    on="SK_ID_BUREAU",
    how="left"
)

# ============================================================
# 3. AGREGAR A NIVEL CLIENTE (créditos → cliente)
# ============================================================

bb_client_features = (
    bb_with_client
    .groupby("SK_ID_CURR")
    .agg({
        "bb_avg_months_past_due": ["mean", "max"],
        "bb_max_months_past_due": ["max"]
    })
)

# Aplanar columnas y dejar nombres finales claros
bb_client_features.columns = [
    "bureau_avg_months_past_due",
    "bureau_max_avg_months_past_due",
    "bureau_max_months_past_due"
]

bb_client_features = bb_client_features.reset_index()

# ============================================================
# RESULTADO FINAL
# ============================================================

print("Features finales generadas desde bureau_balance:")
bb_client_features.head()


In [ ]:
data_set_base = data_set_base.merge(
    bb_client_features,
    on="SK_ID_CURR",
    how="left"
)


En esta etapa se trabajó con la tabla `bureau_balance`, la cual contiene el historial mensual del estado de pago de los créditos externos asociados a los clientes. Dado que esta tabla presenta una granularidad temporal (múltiples registros mensuales por crédito), no puede integrarse directamente al dataset base, cuya granularidad es a nivel cliente.

### 1. Transformación del historial mensual a nivel crédito

El primer paso consistió en transformar la información mensual en métricas resumidas a nivel de crédito (`SK_ID_BUREAU`). Para ello, se utilizó una función genérica de preparación de tablas secundarias, la cual permitió aplicar transformaciones y agregaciones de forma sistemática.

La variable `STATUS`, originalmente categórica, fue convertida a una escala numérica ordinal que representa el número de meses de atraso, donde valores mayores indican mayor severidad del incumplimiento. Los estados correspondientes a créditos cerrados (`C`) o sin información (`X`) fueron considerados como ausencia de atraso.

La variable `MONTHS_BALANCE` fue eliminada deliberadamente, ya que su rol es únicamente temporal y pierde relevancia analítica una vez que el comportamiento histórico del crédito es resumido mediante agregaciones estadísticas.

A nivel de crédito, se generaron métricas que capturan:
- El atraso promedio histórico del crédito.
- El peor atraso registrado durante su historial.

Estas métricas permiten representar el comportamiento de pago de cada crédito de forma compacta y cuantificable.

### 2. Asociación de créditos con clientes

Dado que la tabla `bureau_balance` no contiene el identificador del cliente (`SK_ID_CURR`), fue necesario asociar cada crédito con su respectivo cliente utilizando la tabla `bureau`, la cual actúa como tabla puente entre créditos y clientes.

Este paso es fundamental para mantener la coherencia de la granularidad y permitir la posterior agregación a nivel cliente.

### 3. Agregación de créditos a nivel cliente

Una vez asociados los créditos a sus clientes, se realizó una agregación final a nivel de cliente (`SK_ID_CURR`). En esta etapa, se resumió el comportamiento histórico de todos los créditos externos de cada cliente, generando variables que capturan:

- El atraso promedio considerando todos los créditos externos.
- El peor promedio de atraso entre los créditos.
- El peor atraso histórico absoluto registrado.

El resultado de este proceso es un conjunto de variables numéricas interpretables, con granularidad a nivel cliente, listas para ser integradas al dataset base del proyecto.

### Resultado

Como resultado final, se obtuvo un dataset de features agregadas provenientes del buró de crédito, alineado con la granularidad del dataset principal y diseñado para enriquecer el modelado del riesgo crediticio en etapas posteriores del proceso CRISP-DM.


# Preparación de bureau

In [ ]:
# ============================================================
# 1. TRANSFORMACIÓN DE bureau
# Nivel: crédito (SK_ID_BUREAU)
# ============================================================

features_bureau = preparar_tabla_secundaria(
    df=bureau,
    id_nivel_1="SK_ID_BUREAU",
    cols_drop_logicas=[
        # Identificadores y columnas no informativas para el modelado
        "SK_ID_CURR",  # se reincorpora después
        "CREDIT_CURRENCY",  # alta cardinalidad, poco aporte
    ],
    transformaciones={
        # Días negativos → años positivos
        "DAYS_CREDIT": lambda s: -s / 365,
        "DAYS_CREDIT_ENDDATE": lambda s: -s / 365,
        "DAYS_ENDDATE_FACT": lambda s: -s / 365,
        # Indicador binario de crédito activo
        "CREDIT_ACTIVE": lambda s: s.map(
            {"Active": 1, "Closed": 0, "Sold": 0, "Bad debt": 1}
        ),
    },
    agregaciones_nivel_1={
        # Monto y estado del crédito
        "AMT_CREDIT_SUM": ["mean", "max"],
        "AMT_CREDIT_SUM_DEBT": ["mean", "max"],
        "AMT_CREDIT_SUM_OVERDUE": ["max"],
        "CREDIT_DAY_OVERDUE": ["max"],
        "CREDIT_ACTIVE": ["mean"],
        # Antigüedad del crédito
        "DAYS_CREDIT": ["mean", "max"],
        "DAYS_CREDIT_ENDDATE": ["mean"],
    },
    prefijo="b_",
)

# Renombrar columnas a nombres semánticos (<origen>_<fenómeno>_<resumen>)
features_bureau = features_bureau.rename(
    columns={
        "b_AMT_CREDIT_SUM_mean": "b_avg_credit_amount",
        "b_AMT_CREDIT_SUM_max": "b_max_credit_amount",
        "b_AMT_CREDIT_SUM_DEBT_mean": "b_avg_credit_debt",
        "b_AMT_CREDIT_SUM_DEBT_max": "b_max_credit_debt",
        "b_AMT_CREDIT_SUM_OVERDUE_max": "b_max_credit_overdue",
        "b_CREDIT_DAY_OVERDUE_max": "b_max_days_overdue",
        "b_CREDIT_ACTIVE_mean": "b_share_active_credits",
        "b_DAYS_CREDIT_mean": "b_avg_credit_age_years",
        "b_DAYS_CREDIT_max": "b_max_credit_age_years",
        "b_DAYS_CREDIT_ENDDATE_mean": "b_avg_remaining_term_years",
    }
)

# ============================================================
# 2. ASOCIAR CADA CRÉDITO CON SU CLIENTE (JOIN con bureau)
# ============================================================

b_with_client = features_bureau.merge(
    bureau[["SK_ID_BUREAU", "SK_ID_CURR"]], on="SK_ID_BUREAU", how="left"
)

# ============================================================
# 3. AGREGAR A NIVEL CLIENTE (créditos → cliente)
# ============================================================

b_client_features = b_with_client.groupby("SK_ID_CURR").agg(
    {
        # Montos de crédito
        "b_avg_credit_amount": ["mean", "max"],
        "b_max_credit_amount": ["max"],
        # Deuda
        "b_avg_credit_debt": ["mean", "max"],
        "b_max_credit_debt": ["max"],
        # Atrasos
        "b_max_credit_overdue": ["max"],
        "b_max_days_overdue": ["max"],
        # Estado de créditos
        "b_share_active_credits": ["mean"],
        # Antigüedad
        "b_avg_credit_age_years": ["mean"],
        "b_max_credit_age_years": ["max"],
        # Horizonte restante
        "b_avg_remaining_term_years": ["mean"],
    }
)

# Aplanar columnas y dejar nombres finales claros
b_client_features.columns = [
    "bureau_avg_credit_amount",
    "bureau_max_avg_credit_amount",
    "bureau_max_credit_amount",
    "bureau_avg_credit_debt",
    "bureau_max_avg_credit_debt",
    "bureau_max_credit_debt",
    "bureau_max_credit_overdue",
    "bureau_max_days_overdue",
    "bureau_share_active_credits",
    "bureau_avg_credit_age_years",
    "bureau_max_credit_age_years",
    "bureau_avg_remaining_term_years",
]

b_client_features = b_client_features.reset_index()

# ============================================================
# RESULTADO FINAL
# ============================================================

print("Features finales generadas desde bureau_balance:")
b_client_features.head()

In [ ]:
data_set_base = data_set_base.merge( 
    b_client_features, 
    on="SK_ID_CURR", 
    how="left"
)

La tabla `bureau` contiene el historial de créditos externos de cada cliente reportados por el buró de crédito.  
Su estructura es de **nivel crédito**, donde un mismo cliente (`SK_ID_CURR`) puede tener múltiples registros asociados a distintos créditos (`SK_ID_BUREAU`).

El objetivo de este procesamiento es **transformar esta información transaccional** en un conjunto de **variables agregadas a nivel cliente**, que capturen el comportamiento histórico de endeudamiento, atraso y exposición crediticia, de forma compatible con el dataset base (`application_train`).

---

### 1. Transformación a nivel crédito (`SK_ID_BUREAU`)

#### Eliminación de columnas por lógica estructural

Se eliminaron columnas que no aportan información relevante para el modelado:

- `SK_ID_CURR`:  
  Se elimina temporalmente para evitar duplicación, ya que será reincorporado posteriormente en el proceso de agregación a nivel cliente.

- `CREDIT_CURRENCY`:  
  Presenta alta cardinalidad y no aporta señal relevante en un contexto donde los montos ya se encuentran normalizados. Su inclusión incrementaría el ruido y la dimensionalidad sin beneficio claro.

---

#### Transformaciones aplicadas

#### Conversión de variables temporales

Las siguientes variables, originalmente expresadas como días negativos respecto al momento de la solicitud, se transformaron a **años positivos** para facilitar su interpretación y agregación:

- `DAYS_CREDIT`
- `DAYS_CREDIT_ENDDATE`
- `DAYS_ENDDATE_FACT`

Esta transformación permite interpretar directamente las variables como:
- antigüedad del crédito
- plazo restante estimado
- tiempo transcurrido desde el cierre

---

#### Codificación del estado del crédito

La variable categórica `CREDIT_ACTIVE` se transformó en un indicador binario:

- Créditos activos → `1`
- Créditos cerrados o vendidos → `0`

Esta codificación permite calcular posteriormente la **proporción de créditos activos por cliente**, una señal relevante de exposición crediticia actual.

---

### 2. Agregación a nivel crédito

Una vez transformadas las variables, se realizó una agregación a nivel `SK_ID_BUREAU` para resumir cada crédito mediante estadísticas representativas:

#### Variables agregadas

- **Montos de crédito**
  - Promedio y máximo del monto total del crédito
  - Promedio y máximo de la deuda pendiente

- **Atrasos**
  - Máximo monto en mora
  - Máximo número de días en atraso

- **Estado del crédito**
  - Proporción de tiempo en que el crédito se encuentra activo

- **Antigüedad y horizonte**
  - Antigüedad promedio y máxima del crédito
  - Plazo restante promedio estimado

Estas métricas permiten capturar tanto la **intensidad** como la **persistencia** del comportamiento crediticio.

---

### 3. Asociación crédito → cliente

Dado que la tabla `bureau` se encuentra a nivel crédito, fue necesario realizar un `JOIN` con la tabla original `bureau` para reincorporar la clave `SK_ID_CURR`.

Este paso permite vincular cada crédito resumido con su cliente correspondiente y habilita la agregación final a nivel cliente.

---

### 4. Agregación final a nivel cliente (`SK_ID_CURR`)

Una vez asociados los créditos a sus clientes, se realizó una segunda agregación para obtener variables **a nivel cliente**, tales como:

- Endeudamiento promedio y máximo histórico
- Deuda promedio y máxima
- Máximo atraso histórico registrado
- Proporción de créditos activos
- Antigüedad promedio y máxima del historial crediticio
- Horizonte promedio de los créditos vigentes

Las variables finales siguen el esquema de nombres:



# Preparación de previous_application

In [ ]:
# ============================================================
# 1. TRANSFORMACIÓN DE previous_application
# Nivel: solicitud previa (SK_ID_PREV)
# ============================================================

features_prev = preparar_tabla_secundaria(
    df=previous_application,
    id_nivel_1="SK_ID_PREV",
    cols_drop_logicas=[
        # Identificadores redundantes
        "SK_ID_CURR",  # se reincorpora después

        # Variables con >40% nulos o ruido documental
        "RATE_INTEREST_PRIVILEGED",
        "RATE_INTEREST_PRIMARY",
        "NAME_TYPE_SUITE",

        # Variables de fechas redundantes (muy incompletas)
        "DAYS_FIRST_DRAWING",
        "DAYS_FIRST_DUE",
        "DAYS_LAST_DUE_1ST_VERSION",
        "DAYS_LAST_DUE",
        "DAYS_TERMINATION",
    ],
    transformaciones={
        # Días negativos → años positivos
        "DAYS_DECISION": lambda s: -s / 365,

        # Estado de la solicitud previa
        "NAME_CONTRACT_STATUS": lambda s: s.map({
            "Approved": 1,
            "Refused": 0,
            "Canceled": 0,
            "Unused offer": 0
        }),

        # Indicador de crédito aprobado
        "AMT_CREDIT": lambda s: s.fillna(0),
        "AMT_APPLICATION": lambda s: s.fillna(0),
    },
    agregaciones_nivel_1={
        # Montos solicitados y aprobados
        "AMT_APPLICATION": ["mean", "max"],
        "AMT_CREDIT": ["mean", "max"],
        "AMT_ANNUITY": ["mean", "max"],

        # Estado de aprobación
        "NAME_CONTRACT_STATUS": ["mean"],

        # Antigüedad de solicitudes
        "DAYS_DECISION": ["mean", "min"],

        # Down payment
        "AMT_DOWN_PAYMENT": ["mean", "max"],
    },
    prefijo="pa_",
)

# Renombrar columnas a nombres semánticos (<origen>_<fenómeno>_<resumen>)
features_prev = features_prev.rename(columns={
    "pa_AMT_APPLICATION_mean": "pa_avg_amount_applied",
    "pa_AMT_APPLICATION_max": "pa_max_amount_applied",

    "pa_AMT_CREDIT_mean": "pa_avg_amount_approved",
    "pa_AMT_CREDIT_max": "pa_max_amount_approved",

    "pa_AMT_ANNUITY_mean": "pa_avg_annuity",
    "pa_AMT_ANNUITY_max": "pa_max_annuity",

    "pa_NAME_CONTRACT_STATUS_mean": "pa_approval_rate",

    "pa_DAYS_DECISION_mean": "pa_avg_years_since_decision",
    "pa_DAYS_DECISION_min": "pa_most_recent_years_since_decision",

    "pa_AMT_DOWN_PAYMENT_mean": "pa_avg_down_payment",
    "pa_AMT_DOWN_PAYMENT_max": "pa_max_down_payment",
})

# ============================================================
# 2. ASOCIAR CADA SOLICITUD CON SU CLIENTE (JOIN)
# ============================================================

pa_with_client = features_prev.merge(
    previous_application[["SK_ID_PREV", "SK_ID_CURR"]],
    on="SK_ID_PREV",
    how="left"
)

# ============================================================
# 3. AGREGAR A NIVEL CLIENTE (solicitudes → cliente)
# ============================================================

pa_client_features = pa_with_client.groupby("SK_ID_CURR").agg({
    # Montos
    "pa_avg_amount_applied": ["mean"],
    "pa_max_amount_applied": ["max"],
    "pa_avg_amount_approved": ["mean"],
    "pa_max_amount_approved": ["max"],

    # Anualidades
    "pa_avg_annuity": ["mean"],
    "pa_max_annuity": ["max"],

    # Aprobaciones
    "pa_approval_rate": ["mean"],

    # Antigüedad
    "pa_avg_years_since_decision": ["mean"],
    "pa_most_recent_years_since_decision": ["min"],

    # Down payment
    "pa_avg_down_payment": ["mean"],
    "pa_max_down_payment": ["max"],
})

# Aplanar columnas
pa_client_features.columns = [
    "prev_avg_amount_applied",
    "prev_max_amount_applied",
    "prev_avg_amount_approved",
    "prev_max_amount_approved",
    "prev_avg_annuity",
    "prev_max_annuity",
    "prev_approval_rate",
    "prev_avg_years_since_decision",
    "prev_most_recent_years_since_decision",
    "prev_avg_down_payment",
    "prev_max_down_payment",
]

pa_client_features = pa_client_features.reset_index()

# ============================================================
# RESULTADO FINAL
# ============================================================

print("Features finales generadas desde previous_application:")
pa_client_features.head()


In [ ]:
data_set_base = data_set_base.merge( 
    pa_client_features, 
    on="SK_ID_CURR", 
    how="left"
)

El objetivo de esta etapa es **incorporar información histórica de solicitudes de crédito previas** realizadas por cada cliente, permitiendo capturar patrones de comportamiento crediticio anteriores a la solicitud actual.  
Esta información es especialmente relevante para identificar señales tempranas de riesgo, reincidencia en rechazos, montos solicitados previamente y comportamiento financiero histórico.

---

### Nivel original de la tabla

La tabla `previous_application` se encuentra originalmente a **nivel de solicitud previa**, identificada por la clave:

- `SK_ID_PREV` → identificador único de cada solicitud histórica

Un mismo cliente (`SK_ID_CURR`) puede tener **múltiples solicitudes previas**, por lo que la tabla presenta una relación **uno a muchos** respecto al cliente.

---

### Transformación y preparación

Durante la fase de transformación se realizaron las siguientes acciones clave:

- **Eliminación de columnas no informativas o ruidosas**, tales como identificadores que no aportan señal directa al modelo.
- **Conversión de variables temporales**, transformando días negativos en unidades interpretables (años).
- **Normalización de estados categóricos**, permitiendo su uso en procesos de agregación.
- **Selección de variables financieras relevantes**, priorizando montos solicitados, montos otorgados, diferencias entre ambos y estados de aprobación.

Estas transformaciones permiten homogenizar la información y preparar la tabla para una agregación consistente.

---

### Agregación a nivel cliente

Dado que el modelado se realiza a nivel cliente, las solicitudes previas fueron **agregadas desde el nivel `SK_ID_PREV` al nivel `SK_ID_CURR`**, generando estadísticas resumen que describen el historial del cliente, tales como:

- Promedios y máximos de montos solicitados y aprobados.
- Indicadores de frecuencia de aprobaciones y rechazos.
- Señales de comportamiento financiero previo acumulado.

Este proceso consolida múltiples solicitudes históricas en un único vector de características por cliente.


# Preparación de pos_cash

In [ ]:
# ============================================================
# 1. TRANSFORMACIÓN DE pos_cash_balance
# Nivel: crédito de consumo (SK_ID_PREV)
# ============================================================

features_pos_cash = preparar_tabla_secundaria(
    df=pos_cash,
    id_nivel_1="SK_ID_PREV",
    cols_drop_logicas=[
        "SK_ID_CURR",          # se reincorpora después
        "MONTHS_BALANCE"       # eje temporal, no aporta señal directa al resumir
    ],
    transformaciones={
        # Estado del contrato: indicador de atraso o problemas
        "NAME_CONTRACT_STATUS": lambda s: s.map({
            "Active": 1,
            "Completed": 0,
            "Signed": 1,
            "Returned to the store": 1,
            "Canceled": 1,
            "Approved": 0,
            "Demand": 1
        })
    },
    agregaciones_nivel_1={
        # Atrasos
        "SK_DPD": ["mean", "max"],
        "SK_DPD_DEF": ["mean", "max"],

        # Cuotas
        "CNT_INSTALMENT": ["mean", "max"],
        "CNT_INSTALMENT_FUTURE": ["mean"],

        # Estado del contrato
        "NAME_CONTRACT_STATUS": ["mean"]
    },
    prefijo="pc_"
)

# Renombrar columnas a nombres semánticos (<origen>_<fenómeno>_<resumen>)
features_pos_cash = features_pos_cash.rename(columns={
    "pc_SK_DPD_mean": "pc_avg_days_past_due",
    "pc_SK_DPD_max": "pc_max_days_past_due",

    "pc_SK_DPD_DEF_mean": "pc_avg_days_past_due_def",
    "pc_SK_DPD_DEF_max": "pc_max_days_past_due_def",

    "pc_CNT_INSTALMENT_mean": "pc_avg_installments",
    "pc_CNT_INSTALMENT_max": "pc_max_installments",

    "pc_CNT_INSTALMENT_FUTURE_mean": "pc_avg_future_installments",

    "pc_NAME_CONTRACT_STATUS_mean": "pc_share_problematic_contracts"
})

# ============================================================
# 2. ASOCIAR CADA CRÉDITO CON SU CLIENTE (JOIN con previous_application)
# ============================================================

pc_with_client = features_pos_cash.merge(
    previous_application[["SK_ID_PREV", "SK_ID_CURR"]],
    on="SK_ID_PREV",
    how="left"
)

# ============================================================
# 3. AGREGAR A NIVEL CLIENTE (créditos → cliente)
# ============================================================

pc_client_features = (
    pc_with_client
    .groupby("SK_ID_CURR")
    .agg({
        # Atrasos
        "pc_avg_days_past_due": ["mean", "max"],
        "pc_max_days_past_due": ["max"],

        "pc_avg_days_past_due_def": ["mean", "max"],
        "pc_max_days_past_due_def": ["max"],

        # Cuotas
        "pc_avg_installments": ["mean"],
        "pc_max_installments": ["max"],
        "pc_avg_future_installments": ["mean"],

        # Estado contractual
        "pc_share_problematic_contracts": ["mean"]
    })
)

# Aplanar columnas y dejar nombres finales claros
pc_client_features.columns = [
    "pos_avg_days_past_due",
    "pos_max_avg_days_past_due",
    "pos_max_days_past_due",

    "pos_avg_days_past_due_def",
    "pos_max_avg_days_past_due_def",
    "pos_max_days_past_due_def",

    "pos_avg_installments",
    "pos_max_installments",
    "pos_avg_future_installments",

    "pos_share_problematic_contracts"
]

pc_client_features = pc_client_features.reset_index()

# ============================================================
# RESULTADO FINAL
# ============================================================

print("Features finales generadas desde POS_CASH_balance:")
pc_client_features.head()


In [ ]:
data_set_base = data_set_base.merge(
    pc_client_features,
    on="SK_ID_CURR",
    how="left"
)

La tabla `POS_CASH_balance` contiene información histórica mensual sobre créditos de tipo **POS / consumo**, asociados a solicitudes previas (`SK_ID_PREV`). Su estructura es temporal, con múltiples registros por crédito y cliente, por lo que **no puede incorporarse directamente al dataset base sin una agregación previa**.

El objetivo de este procesamiento es **resumir el comportamiento de pago histórico de los créditos POS** y transformarlo en variables informativas a nivel cliente (`SK_ID_CURR`).

---

### 1. Eliminación de columnas no informativas

- **`MONTHS_BALANCE`**  
  Representa el eje temporal relativo (meses antes del corte). Al resumir el comportamiento completo del crédito, esta variable no aporta señal directa al modelo y se elimina.

- **`SK_ID_CURR`**  
  Se elimina temporalmente para evitar duplicidad, ya que la asociación cliente–crédito se realiza posteriormente mediante `previous_application`.

---

### 2. Transformación de variables clave

#### Estado del contrato (`NAME_CONTRACT_STATUS`)
Se transforma a un **indicador binario de riesgo operativo**, donde:
- Valores como `Active`, `Signed`, `Returned`, `Canceled` y `Demand` se consideran **potencialmente problemáticos**.
- Estados como `Completed` y `Approved` se consideran **no problemáticos**.

Esto permite capturar la **proporción de créditos POS con incidencias** por cliente.

---

### 3. Agregación a nivel crédito (`SK_ID_PREV`)

Cada crédito POS puede tener múltiples registros mensuales. Por ello, se generan resúmenes estadísticos que caracterizan su comportamiento global:

#### Atrasos
- Promedio y máximo de días de atraso (`SK_DPD`)
- Promedio y máximo de atraso con default (`SK_DPD_DEF`)

#### Estructura de cuotas
- Número promedio y máximo de cuotas
- Cuotas futuras promedio pendientes

#### Estado contractual
- Proporción del tiempo en estados contractuales problemáticos

Estas métricas permiten capturar **frecuencia, severidad y persistencia del incumplimiento** a nivel crédito.

---

### 4. Asociación crédito → cliente

Los créditos POS (`SK_ID_PREV`) se vinculan con su cliente correspondiente (`SK_ID_CURR`) mediante la tabla `previous_application`.  
Este paso es fundamental para consolidar la información a nivel cliente.

---

### 5. Agregación final a nivel cliente (`SK_ID_CURR`)

Finalmente, los créditos POS de cada cliente se agregan para obtener indicadores globales de comportamiento:

#### Indicadores de atraso
- Atraso promedio histórico
- Peor atraso promedio observado
- Peor atraso máximo registrado

#### Indicadores de estructura de pago
- Número promedio de cuotas
- Máximo número de cuotas
- Promedio de cuotas futuras pendientes

#### Indicador de riesgo operativo
- Proporción de créditos POS con estados contractuales problemáticos

---

### Resultado

El resultado es un conjunto de variables agregadas a nivel cliente que describen el **historial de comportamiento en créditos de consumo POS**, las cuales se integran al `data_set_base` mediante un `LEFT JOIN` sobre `SK_ID_CURR`.

Estas variables aportan información complementaria relevante para el análisis de riesgo, especialmente en clientes con múltiples créditos de corto plazo.


# Preparación de installments

In [ ]:
# ============================================================
# 1. TRANSFORMACIÓN DE installments_payments
# Nivel: cuota (registro individual)
# ============================================================

installments_proc = installments.copy()

# ------------------------------------------------------------
# Variables derivadas clave
# ------------------------------------------------------------

# Diferencia entre lo pagado y lo que debía pagarse
installments_proc["inst_payment_diff"] = (
    installments_proc["AMT_PAYMENT"] - installments_proc["AMT_INSTALMENT"]
)

# Retraso en días (positivo = pago tardío)
installments_proc["inst_days_late"] = (
    installments_proc["DAYS_ENTRY_PAYMENT"] - installments_proc["DAYS_INSTALMENT"]
)

# Indicadores binarios
installments_proc["inst_late_flag"] = (installments_proc["inst_days_late"] > 0).astype(int)
installments_proc["inst_underpay_flag"] = (installments_proc["inst_payment_diff"] < 0).astype(int)

# ============================================================
# 2. AGREGACIÓN A NIVEL CRÉDITO (SK_ID_PREV)
# ============================================================

inst_credit_features = (
    installments_proc
    .groupby("SK_ID_PREV")
    .agg({
        # Atrasos
        "inst_days_late": ["mean", "max"],
        "inst_late_flag": ["mean"],

        # Montos
        "inst_payment_diff": ["mean", "min"],
        "inst_underpay_flag": ["mean"]
    })
)

# Aplanar columnas
inst_credit_features.columns = [
    "inst_avg_days_late",
    "inst_max_days_late",
    "inst_share_late_installments",
    "inst_avg_payment_diff",
    "inst_min_payment_diff",
    "inst_share_underpaid_installments"
]

inst_credit_features = inst_credit_features.reset_index()

# ============================================================
# 3. ASOCIAR CRÉDITO CON CLIENTE (JOIN con previous_application)
# ============================================================

inst_with_client = inst_credit_features.merge(
    previous_application[["SK_ID_PREV", "SK_ID_CURR"]],
    on="SK_ID_PREV",
    how="left"
)

# ============================================================
# 4. AGREGACIÓN FINAL A NIVEL CLIENTE (SK_ID_CURR)
# ============================================================

inst_client_features = (
    inst_with_client
    .groupby("SK_ID_CURR")
    .agg({
        # Atrasos
        "inst_avg_days_late": ["mean", "max"],
        "inst_max_days_late": ["max"],
        "inst_share_late_installments": ["mean"],

        # Comportamiento de pago
        "inst_avg_payment_diff": ["mean"],
        "inst_min_payment_diff": ["min"],
        "inst_share_underpaid_installments": ["mean"]
    })
)

# Aplanar columnas con nombres semánticos
inst_client_features.columns = [
    "inst_avg_days_late",
    "inst_max_avg_days_late",
    "inst_max_days_late",
    "inst_share_late_installments",
    "inst_avg_payment_diff",
    "inst_worst_payment_diff",
    "inst_share_underpaid_installments"
]

inst_client_features = inst_client_features.reset_index()

# ============================================================
# RESULTADO FINAL
# ============================================================

print("Features finales generadas desde installments_payments:")
inst_client_features.head()


In [ ]:
data_set_base = data_set_base.merge(
    inst_client_features,
    on="SK_ID_CURR",
    how="left"
)

La tabla `installments_payments` contiene el historial detallado de pagos de cuotas asociadas a créditos previos (`SK_ID_PREV`). Cada registro representa **una cuota individual**, lo que genera múltiples observaciones por crédito y cliente.

Debido a su granularidad transaccional, esta tabla no puede integrarse directamente al `data_set_base`. Es necesario transformar y resumir la información para capturar patrones de **disciplina de pago**, **retrasos** y **cumplimiento financiero** a nivel cliente (`SK_ID_CURR`).

---

### 1. Generación de variables derivadas

Con el objetivo de transformar fechas y montos en señales explícitas de comportamiento financiero, se crean las siguientes variables:

#### Diferencia entre monto pagado y monto comprometido
- **`inst_payment_diff = AMT_PAYMENT - AMT_INSTALMENT`**  
  Valores negativos indican subpago; valores positivos representan pagos adelantados o superiores al mínimo.

#### Retraso en días
- **`inst_days_late = DAYS_ENTRY_PAYMENT - DAYS_INSTALMENT`**  
  Valores positivos reflejan pagos tardíos; valores negativos indican pagos anticipados.

#### Indicadores binarios
- **`inst_late_flag`**: identifica cuotas pagadas fuera de plazo.
- **`inst_underpay_flag`**: identifica cuotas pagadas por debajo del monto comprometido.

Estas transformaciones permiten convertir información operacional en **variables directamente interpretables por modelos de riesgo**.

---

### 2. Agregación a nivel crédito (`SK_ID_PREV`)

Cada crédito puede contener múltiples cuotas. Para caracterizar el comportamiento completo del crédito, se generan agregaciones estadísticas que resumen:

#### Atrasos
- Atraso promedio en días.
- Peor atraso observado.
- Proporción de cuotas pagadas con atraso.

#### Comportamiento de pago
- Diferencia promedio entre monto pagado y monto comprometido.
- Peor subpago observado.
- Proporción de cuotas pagadas de forma incompleta.

Estas métricas permiten distinguir entre **incumplimientos puntuales** y **patrones sistemáticos de mala conducta de pago**.

---

### 3. Asociación crédito → cliente

Los créditos (`SK_ID_PREV`) se vinculan con su cliente correspondiente (`SK_ID_CURR`) utilizando la tabla `previous_application`.  
Este paso consolida múltiples créditos bajo una misma entidad cliente, habilitando el análisis a nivel individuo.

---

### 4. Agregación final a nivel cliente (`SK_ID_CURR`)

Los créditos de cada cliente se agregan para obtener indicadores globales de comportamiento financiero:

#### Indicadores de puntualidad
- Atraso promedio histórico del cliente.
- Peor atraso promedio entre sus créditos.
- Peor atraso máximo registrado.

#### Indicadores de disciplina de pago
- Proporción promedio de cuotas pagadas tarde.
- Diferencia promedio entre pago y cuota.
- Peor subpago observado.
- Proporción de cuotas pagadas de forma incompleta.

---

### Resultado

El resultado es un conjunto de variables agregadas a nivel cliente que describen **la consistencia, puntualidad y calidad del comportamiento de pago en cuotas**.

Estas variables se integran al `data_set_base` mediante un `LEFT JOIN` sobre `SK_ID_CURR`, aportando una de las señales más fuertes para la evaluación del riesgo crediticio en el modelo final.


# Preparación de credit_card

In [ ]:
# ============================================================
# 1. TRANSFORMACIÓN DE credit_card_balance
# Nivel: mes de tarjeta de crédito
# ============================================================

cc_proc = credit_card.copy()

# ------------------------------------------------------------
# Variables derivadas clave
# ------------------------------------------------------------

# Utilización de la línea de crédito
cc_proc["cc_utilization"] = (
    cc_proc["AMT_BALANCE"] / cc_proc["AMT_CREDIT_LIMIT_ACTUAL"]
)

# Indicador de atraso
cc_proc["cc_late_flag"] = (cc_proc["SK_DPD"] > 0).astype(int)
cc_proc["cc_late_def_flag"] = (cc_proc["SK_DPD_DEF"] > 0).astype(int)

# ------------------------------------------------------------
# Manejo de valores extremos
# ------------------------------------------------------------

# Reemplazar divisiones inválidas
cc_proc["cc_utilization"] = cc_proc["cc_utilization"].replace([np.inf, -np.inf], np.nan)

# ============================================================
# 2. AGREGACIÓN A NIVEL TARJETA / CRÉDITO (SK_ID_PREV)
# ============================================================

cc_credit_features = (
    cc_proc
    .groupby("SK_ID_PREV")
    .agg({
        # Utilización
        "cc_utilization": ["mean", "max"],

        # Atrasos
        "SK_DPD": ["mean", "max"],
        "SK_DPD_DEF": ["max"],
        "cc_late_flag": ["mean"],
        "cc_late_def_flag": ["mean"],

        # Montos
        "AMT_BALANCE": ["mean", "max"],
        "AMT_DRAWINGS_CURRENT": ["mean"],
        "AMT_PAYMENT_CURRENT": ["mean"]
    })
)

# Aplanar columnas
cc_credit_features.columns = [
    "cc_avg_utilization",
    "cc_max_utilization",
    "cc_avg_days_late",
    "cc_max_days_late",
    "cc_max_days_late_def",
    "cc_share_late_months",
    "cc_share_late_def_months",
    "cc_avg_balance",
    "cc_max_balance",
    "cc_avg_drawings",
    "cc_avg_payment"
]

cc_credit_features = cc_credit_features.reset_index()

# ============================================================
# 3. ASOCIAR TARJETA CON CLIENTE (JOIN con previous_application)
# ============================================================

cc_with_client = cc_credit_features.merge(
    previous_application[["SK_ID_PREV", "SK_ID_CURR"]],
    on="SK_ID_PREV",
    how="left"
)

# ============================================================
# 4. AGREGACIÓN FINAL A NIVEL CLIENTE (SK_ID_CURR)
# ============================================================

cc_client_features = (
    cc_with_client
    .groupby("SK_ID_CURR")
    .agg({
        # Utilización
        "cc_avg_utilization": ["mean", "max"],
        "cc_max_utilization": ["max"],

        # Atrasos
        "cc_avg_days_late": ["mean"],
        "cc_max_days_late": ["max"],
        "cc_max_days_late_def": ["max"],
        "cc_share_late_months": ["mean"],
        "cc_share_late_def_months": ["mean"],

        # Montos
        "cc_avg_balance": ["mean"],
        "cc_max_balance": ["max"],
        "cc_avg_drawings": ["mean"],
        "cc_avg_payment": ["mean"]
    })
)

# Aplanar columnas con nombres semánticos
cc_client_features.columns = [
    "cc_avg_utilization",
    "cc_max_avg_utilization",
    "cc_max_utilization",
    "cc_avg_days_late",
    "cc_max_days_late",
    "cc_max_days_late_def",
    "cc_share_late_months",
    "cc_share_late_def_months",
    "cc_avg_balance",
    "cc_max_balance",
    "cc_avg_drawings",
    "cc_avg_payment"
]

cc_client_features = cc_client_features.reset_index()

# ============================================================
# RESULTADO FINAL
# ============================================================

print("Features finales generadas desde credit_card_balance:")
cc_client_features.head()


In [ ]:
data_set_base = data_set_base.merge(
    cc_client_features,
    on="SK_ID_CURR",
    how="left"
)

La tabla `credit_card_balance` contiene información histórica mensual sobre el uso de tarjetas de crédito asociadas a créditos previos (`SK_ID_PREV`). Cada registro representa el estado de una tarjeta en un mes determinado, incluyendo saldo, límite, pagos, giros y atrasos.

Debido a su naturaleza temporal y transaccional, esta tabla no puede incorporarse directamente al `data_set_base`. Es necesario resumir su información para capturar **patrones de utilización del crédito**, **comportamiento de pago** y **severidad de atrasos** a nivel cliente (`SK_ID_CURR`).

---

### 1. Generación de variables derivadas

Con el objetivo de transformar montos y estados en indicadores financieros interpretables, se crean las siguientes variables:

#### Utilización de la línea de crédito
- **`cc_utilization = AMT_BALANCE / AMT_CREDIT_LIMIT_ACTUAL`**  
  Mide el nivel de uso del cupo disponible. Valores altos reflejan mayor presión financiera y riesgo potencial.

Se controla explícitamente la aparición de valores infinitos o indefinidos, reemplazándolos por valores nulos para evitar sesgos en etapas posteriores.

#### Indicadores de atraso
- **`cc_late_flag`**: identifica meses con atraso en el pago.
- **`cc_late_def_flag`**: identifica meses con atraso en default.

Estas variables permiten distinguir entre atrasos leves y atrasos con mayor severidad.

---

### 2. Agregación a nivel tarjeta / crédito (`SK_ID_PREV`)

Cada tarjeta de crédito puede presentar múltiples registros mensuales. Para caracterizar el comportamiento completo de la tarjeta, se generan agregaciones estadísticas que resumen:

#### Utilización del crédito
- Utilización promedio histórica.
- Máxima utilización registrada.

#### Atrasos
- Atraso promedio en días.
- Peor atraso observado.
- Peor atraso en situación de default.
- Proporción de meses con atraso.
- Proporción de meses con atraso en default.

#### Montos financieros
- Saldo promedio y máximo.
- Promedio de giros realizados.
- Promedio de pagos efectuados.

Estas métricas permiten capturar tanto la **intensidad del uso del crédito** como la **consistencia del comportamiento de pago**.

---

### 3. Asociación tarjeta → cliente

Las tarjetas (`SK_ID_PREV`) se vinculan con su cliente correspondiente (`SK_ID_CURR`) utilizando la tabla `previous_application`.  
Este paso consolida la información de múltiples tarjetas bajo una misma entidad cliente.

---

### 4. Agregación final a nivel cliente (`SK_ID_CURR`)

Las tarjetas de cada cliente se agregan para generar indicadores globales de comportamiento financiero:

#### Indicadores de utilización
- Utilización promedio histórica del cliente.
- Peor utilización promedio observada.
- Máxima utilización registrada.

#### Indicadores de atraso
- Atraso promedio histórico.
- Peor atraso observado.
- Peor atraso en default.
- Proporción promedio de meses con atraso.
- Proporción de meses con atraso severo.

#### Indicadores de flujo financiero
- Saldo promedio.
- Saldo máximo.
- Promedio de giros realizados.
- Promedio de pagos efectuados.

---

### Resultado

El resultado es un conjunto de variables agregadas a nivel cliente que describen **el uso del crédito rotativo, la presión financiera y la severidad del incumplimiento** en tarjetas de crédito.

Estas variables se integran al `data_set_base` mediante un `LEFT JOIN` sobre `SK_ID_CURR`, complementando la información proveniente de créditos POS, cuotas, buró y solicitudes previas, y cerrando así el conjunto completo de variables financieras históric


# Validación final

In [ ]:
# ============================================================
# RESUMEN GENERAL DEL DATA_SET_BASE
# ============================================================

print("==============================================")
print("RESUMEN GENERAL DEL DATA_SET_BASE")
print("==============================================\n")

# 1. Dimensiones
print(" Dimensiones del dataset")
print(f"Filas (clientes): {data_set_base.shape[0]}")
print(f"Columnas (features): {data_set_base.shape[1]}\n")

# 2. Tipos de datos
print(" Tipos de datos")
display(data_set_base.dtypes.value_counts())

# 3. Resumen de valores nulos
print("\n Resumen de valores nulos (% por columna)")
null_summary = data_set_base.isna().mean().sort_values(ascending=False)

display(null_summary.head(20))

print(f"\nColumnas con al menos un valor nulo: {(null_summary > 0).sum()}")
print(f"Columnas sin nulos: {(null_summary == 0).sum()}")

# 4. Columnas con más del 40% de nulos
print("\n Columnas con más del 40% de valores nulos")
high_null_cols = null_summary[null_summary > 0.40]
display(high_null_cols)

# 5. Chequeo de duplicados
print("\n Chequeo de duplicados")
print("Filas duplicadas:", data_set_base.duplicated().sum())

# 6. Vista general de columnas (ordenadas)
print("\n Lista de columnas (ordenadas alfabéticamente)")
display(pd.Series(data_set_base.columns).sort_values().reset_index(drop=True))

print("\nFin del resumen del data_set_base")

# Eliminación de variables de la tabla application

## Flags de documentos (ruido estructural)

In [ ]:
doc_flags = [col for col in data_set_base.columns if col.startswith("FLAG_DOCUMENT_")]

doc_summary = data_set_base[doc_flags].mean().sort_values(ascending=False)

print("Proporción de valores 1 por FLAG_DOCUMENT:")
display(doc_summary)

In [ ]:
# Eliminación de las columnas FLAG_DOCUMENT
flag_cols_to_drop = [col for col in data_set_base.columns if col.startswith("FLAG_DOCUMENT")]
data_set_base = data_set_base.drop(columns=flag_cols_to_drop)

# Verificación de las dimensiones del dataset después de la eliminación
print("Dimensiones después de eliminar FLAG_DOCUMENT columns:")
print(data_set_base.shape)


#### Eliminación de columnas `FLAG_DOCUMENT_*` (Ruido Estructural)

**Justificación:**

Las columnas `FLAG_DOCUMENT_*` representan indicadores binarios de documentos específicos que han sido asociados a los clientes. Sin embargo, tras analizar las proporciones de valores 1 en cada una de estas columnas, encontramos que la mayoría tienen un porcentaje de valores 0 extremadamente alto, con columnas como `FLAG_DOCUMENT_3` teniendo un 71% de 1s, y las demás en su mayoría con valores cercanos a cero. Estas columnas no aportan discriminación útil para la predicción de incumplimiento de pago, ya que el modelo no puede aprender patrones significativos a partir de una mayoría tan abrumadora de ceros.

**Acción:**

Se ha decidido eliminar todas las columnas que comienzan con `FLAG_DOCUMENT_`, ya que su alto nivel de ruido las convierte en irrelevantes para el modelado.


## Columnas constantes (VARIANZA CERO)

In [ ]:
constant_cols = [
    col for col in data_set_base.columns
    if data_set_base[col].nunique(dropna=False) <= 1
]

print("Columnas constantes detectadas:")
print(constant_cols)

print("Cantidad:", len(constant_cols))


## Columnas de baja varianza (VARIANZA ≈ 0)

In [ ]:
numeric_cols = data_set_base.select_dtypes(include=["int64", "float64"]).columns
numeric_data = data_set_base[numeric_cols]

vt = VarianceThreshold(threshold=0.01)
vt.fit(numeric_data)

low_variance_cols = numeric_data.columns[~vt.get_support()]

print("Columnas numéricas con varianza < 0.01:")
print(list(low_variance_cols))

print("Cantidad:", len(low_variance_cols))


In [ ]:
cols_low_variance_drop = [
    # Flags estructurales
    "FLAG_MOBIL",
    "FLAG_CONT_MOBILE",
    "AMT_REQ_CREDIT_BUREAU_HOUR",

    # Variables inmobiliarias redundantes
    "BASEMENTAREA_AVG", "YEARS_BEGINEXPLUATATION_AVG", "COMMONAREA_AVG",
    "LANDAREA_AVG", "LIVINGAPARTMENTS_AVG", "NONLIVINGAPARTMENTS_AVG", "NONLIVINGAREA_AVG",
    "BASEMENTAREA_MODE", "YEARS_BEGINEXPLUATATION_MODE", "COMMONAREA_MODE",
    "LANDAREA_MODE", "LIVINGAPARTMENTS_MODE", "NONLIVINGAPARTMENTS_MODE", "NONLIVINGAREA_MODE",
    "BASEMENTAREA_MEDI", "YEARS_BEGINEXPLUATATION_MEDI", "COMMONAREA_MEDI",
    "LANDAREA_MEDI", "LIVINGAPARTMENTS_MEDI", "NONLIVINGAPARTMENTS_MEDI", "NONLIVINGAREA_MEDI"
]

data_set_base = data_set_base.drop(columns=cols_low_variance_drop, errors="ignore")

print("Dimensiones tras eliminar columnas de baja varianza estructural:")
print(data_set_base.shape)


### Eliminación de columnas de baja varianza

Se identificaron columnas numéricas con varianza inferior a 0.01. Estas variables fueron evaluadas según su naturaleza y origen.

- Se eliminaron variables estructurales y documentales con varianza casi nula, dado que no aportan capacidad discriminante al modelo.
- Se eliminaron variables inmobiliarias en sus versiones AVG, MODE y MEDI, ya que presentan alto porcentaje de valores nulos, redundancia entre sí y baja señal predictiva.
- Se conservaron variables agregadas provenientes de tablas secundarias (POS_CASH y CREDIT_CARD), aun cuando presentan baja varianza, debido a que representan comportamiento financiero histórico y pueden ser relevantes para ciertos subgrupos de clientes.

Esta decisión permite reducir ruido manteniendo variables de alto valor semántico para el modelado.


## Redundancia y multicolinealidad extrema

In [ ]:
corr_matrix = data_set_base[numeric_cols].corr().abs()

high_corr_pairs = []

threshold = 0.85

for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if corr_matrix.iloc[i, j] > threshold:
            col1 = corr_matrix.columns[i]
            col2 = corr_matrix.columns[j]
            high_corr_pairs.append((col1, col2, corr_matrix.iloc[i, j]))

print("Pares con correlación > 0.85:")
for c1, c2, v in high_corr_pairs:
    print(f"{c1} ↔ {c2} → corr={v:.3f}")


In [ ]:
# ------------------------------------------------------------
# 1. Eliminar versiones MODE y MEDI (redundancia estructural)
# ------------------------------------------------------------
cols_drop_mode_medi = [
    col for col in data_set_base.columns
    if col.endswith("_MODE") or col.endswith("_MEDI")
]

# ------------------------------------------------------------
# 2. Redundancias clave de application_train
# ------------------------------------------------------------
cols_drop_application_corr = [
    "AMT_GOODS_PRICE",                 # redundante con AMT_CREDIT
    "CNT_CHILDREN",                    # redundante con CNT_FAM_MEMBERS
    "REGION_RATING_CLIENT_W_CITY",     # redundante con REGION_RATING_CLIENT
    "OBS_60_CNT_SOCIAL_CIRCLE",        # redundante con OBS_30
    "DEF_60_CNT_SOCIAL_CIRCLE"         # redundante con DEF_30
]

# ------------------------------------------------------------
# 3. Redundancias de features creadas (feature engineering)
# ------------------------------------------------------------
cols_drop_engineered = [
    # bureau
    "bureau_max_avg_credit_amount",
    "bureau_max_avg_credit_debt",

    # previous_application
    "prev_avg_amount_applied",
    "prev_max_amount_applied",
    "prev_avg_down_payment",

    # POS CASH
    "pos_max_avg_days_past_due",
    "pos_max_avg_days_past_due_def",

    # credit card
    "cc_max_avg_utilization",
    "cc_avg_days_late"
]

# ------------------------------------------------------------
# 4. Consolidar columnas a eliminar
# ------------------------------------------------------------
cols_to_drop_corr = (
    cols_drop_mode_medi +
    cols_drop_application_corr +
    cols_drop_engineered
)

# ------------------------------------------------------------
# 5. Aplicar eliminación
# ------------------------------------------------------------
data_set_base = data_set_base.drop(columns=cols_to_drop_corr, errors="ignore")

# ------------------------------------------------------------
# 6. Verificación final
# ------------------------------------------------------------
print("Columnas eliminadas por multicolinealidad extrema:", len(cols_to_drop_corr))
print("Dimensiones finales del data_set_base:", data_set_base.shape)

#### Análisis de multicolinealidad extrema

Tras la eliminación de columnas irrelevantes y de baja varianza, se recalculó el conjunto de variables numéricas presentes en el dataset final. Esto permitió evaluar correctamente la multicolinealidad entre variables sin incluir columnas previamente eliminadas.

Se utilizó la matriz de correlación absoluta para identificar pares de variables con correlación superior a 0.85, las cuales indican redundancia extrema. Para cada par detectado, se evaluó la naturaleza de las variables y se mantuvo aquella con mayor interpretabilidad y valor semántico para el modelado.

Este proceso contribuye a mejorar la estabilidad del modelo y a reducir la redundancia innecesaria entre predictores.


## Columnas con alto porcentaje de nulos

In [ ]:
null_ratio = data_set_base.isna().mean().sort_values(ascending=False)

high_null_cols = null_ratio[null_ratio > 0.5]

print("Columnas con más de 50% de valores nulos:")
display(high_null_cols)


In [ ]:
# ------------------------------------------------------------
# 1. IMPUTACIÓN DE NULOS EN VARIABLES DE PRODUCTOS FINANCIEROS
#    (bureau, credit card, POS)
# ------------------------------------------------------------
# En estas variables, el NaN indica ausencia del producto,
# lo cual es información válida → se imputa con 0

cols_financial_nulls = [
    col for col in data_set_base.columns if col.startswith(("cc_", "bureau_", "pos_"))
]

data_set_base[cols_financial_nulls] = data_set_base[cols_financial_nulls].fillna(0)

print(" Imputación con 0 aplicada a variables financieras")
print(
    "Nulos restantes en variables financieras:",
    data_set_base[cols_financial_nulls].isna().sum().sum(),
)


# ------------------------------------------------------------
# 2. ELIMINACIÓN DE VARIABLES FÍSICAS DE VIVIENDA
#    (> 50% de valores nulos)
# ------------------------------------------------------------
cols_housing_high_nulls = [
    "FLOORSMIN_AVG",
    "YEARS_BUILD_AVG",
    "ELEVATORS_AVG",
    "APARTMENTS_AVG",
    "ENTRANCES_AVG",
    "LIVINGAREA_AVG",
]

data_set_base = data_set_base.drop(columns=cols_housing_high_nulls, errors="ignore")

print("\n Variables de vivienda eliminadas por alto porcentaje de nulos:")
print(cols_housing_high_nulls)
print("Dimensiones tras eliminación:", data_set_base.shape)


# ------------------------------------------------------------
# 3. VERIFICACIÓN FINAL DE NULOS
# ------------------------------------------------------------
null_ratio = data_set_base.isna().mean().sort_values(ascending=False)

print("\nTop columnas con mayor proporción de nulos restantes:")
print(null_ratio.head(10))

print("\nTotal de valores nulos restantes en el dataset:")
print(data_set_base.isna().sum().sum())

#### Tratamiento de valores nulos

Se identificaron columnas con altos porcentajes de valores nulos, distinguiendo entre dos tipos de casos:

1. **Variables asociadas a productos financieros** (bureau, credit card, POS):
   En estos casos, los valores nulos representan la ausencia del producto financiero en el cliente. Esta ausencia constituye información relevante para el riesgo crediticio, por lo que dichas variables fueron conservadas e imputadas con valor 0.

2. **Variables físicas de la vivienda**:
   Estas variables presentan más del 50% de valores nulos debido a la falta de información declarada. Dado su bajo aporte predictivo y alta incompletitud, se decidió eliminarlas para evitar ruido y sesgos en el modelado.

Este enfoque permite preservar señales de negocio relevantes y mejorar la calidad general del dataset.


## Eliminación de variables categoricas ruidosas

In [ ]:
cols_drop_categorical = [
    "ORGANIZATION_TYPE",
    "OCCUPATION_TYPE",
    "NAME_TYPE_SUITE",
    "WEEKDAY_APPR_PROCESS_START"
]

data_set_base = data_set_base.drop(columns=cols_drop_categorical, errors="ignore")

print("Columnas categóricas eliminadas:")
print(cols_drop_categorical)
print("Dimensiones tras eliminación:", data_set_base.shape)


En esta etapa se identifican y eliminan variables categóricas que, si bien contienen información descriptiva, **no aportan señal relevante al riesgo crediticio** o generan **ruido estructural** durante el modelado.

La decisión de eliminación se fundamenta en los siguientes criterios:

- **Alta cardinalidad**, que provoca explosión dimensional al aplicar One-Hot Encoding.
- **Alto porcentaje de valores nulos**, dificultando una imputación semánticamente válida.
- **Baja relación causal** con el incumplimiento de pago.
- Variables **operativas o administrativas**, no asociadas al comportamiento financiero del cliente.

#### Variables eliminadas

- **ORGANIZATION_TYPE**  
  Presenta 58 categorías distintas. Su alta cardinalidad genera un gran número de variables dummy sin aportar una señal clara y consistente al modelo.

- **OCCUPATION_TYPE**  
  Contiene 18 categorías y aproximadamente un 31% de valores nulos. La imputación de esta variable es ambigua y su codificación introduce ruido y riesgo de sobreajuste.

- **NAME_TYPE_SUITE**  
  Describe quién acompaña al solicitante durante la postulación. Esta información no tiene relación directa con la capacidad de pago ni con el comportamiento crediticio.

- **WEEKDAY_APPR_PROCESS_START**  
  Corresponde a una variable de proceso interno (día de la semana de la solicitud). No refleja características del cliente ni su riesgo financiero.

#### Resultado del proceso

Tras eliminar estas variables categóricas ruidosas, el dataset reduce su dimensionalidad y complejidad, quedando mejor preparado para:

- El encoding de variables categóricas relevantes.
- El escalado numérico posterior.
- El entrenamiento de modelos más estables y generalizables.

Este paso contribuye directamente a mejorar la **calidad del dataset base** y a reducir el riesgo de sobreajuste en las etapas de modelado.


# Codificación de variables categorias del dataset

## Encoding ordinal

In [45]:

education_map = {
    "Lower secondary": 0,
    "Secondary / secondary special": 1,
    "Incomplete higher": 2,
    "Higher education": 3,
    "Academic degree": 4
}

data_set_base["EDU_LEVEL"] = data_set_base["NAME_EDUCATION_TYPE"].map(education_map)

# Verificar nulos
print("Nulos generados en EDU_LEVEL:", data_set_base["EDU_LEVEL"].isna().sum())

# Eliminar columna original
data_set_base = data_set_base.drop(columns=["NAME_EDUCATION_TYPE"])

Nulos generados en EDU_LEVEL: 0


### Codificación ordinal de nivel educacional

La variable **NAME_EDUCATION_TYPE** representa el nivel educacional alcanzado por el solicitante. A diferencia de otras variables categóricas, esta variable posee un **orden natural y jerárquico**, lo que permite una transformación ordinal sin pérdida semántica.

#### Justificación del enfoque ordinal

- Existe una **relación implícita de progresión** entre los niveles educacionales.
- El nivel educacional está directamente relacionado con:
  - Estabilidad laboral
  - Ingresos potenciales
  - Capacidad de pago
- Aplicar One-Hot Encoding en este caso eliminaría dicha jerarquía y aumentaría innecesariamente la dimensionalidad.

#### Validación del proceso

- Se verificó que el mapeo **no generara valores nulos**, lo que confirma que todas las categorías presentes fueron correctamente contempladas.
- Una vez creada la variable ordinal **EDU_LEVEL**, se eliminó la columna original **NAME_EDUCATION_TYPE** para evitar redundancia.

#### Resultado

El dataset base incorpora ahora una variable numérica interpretable que conserva la semántica original del nivel educacional y mejora la eficiencia del modelado posterior.


## One-Hot Encoding

In [48]:
cols_to_encode = [
    "NAME_CONTRACT_TYPE",
    "CODE_GENDER",
    "FLAG_OWN_CAR",
    "FLAG_OWN_REALTY",
    "NAME_INCOME_TYPE",
    "NAME_FAMILY_STATUS",
    "NAME_HOUSING_TYPE"
]

encoder = OneHotEncoder(drop="first", sparse_output=False)

encoded = encoder.fit_transform(data_set_base[cols_to_encode])

encoded_df = pd.DataFrame(
    encoded,
    columns=encoder.get_feature_names_out(cols_to_encode),
    index=data_set_base.index
)

# Eliminar originales
data_set_base = data_set_base.drop(columns=cols_to_encode)

# Agregar dummies
data_set_base = pd.concat([data_set_base, encoded_df], axis=1)

print("One-Hot Encoding aplicado")
print("Dimensiones actuales:", data_set_base.shape)

One-Hot Encoding aplicado
Dimensiones actuales: (307511, 105)


### Codificación One-Hot de variables categóricas nominales

Tras la eliminación de categóricas ruidosas y la transformación ordinal del nivel educacional, se identificaron variables categóricas **nominales** que no poseen un orden inherente y que, por tanto, requieren un esquema de codificación distinto.

#### Variables codificadas

Se aplicó **One-Hot Encoding** a las siguientes variables:

- `NAME_CONTRACT_TYPE`
- `CODE_GENDER`
- `FLAG_OWN_CAR`
- `FLAG_OWN_REALTY`
- `NAME_INCOME_TYPE`
- `NAME_FAMILY_STATUS`
- `NAME_HOUSING_TYPE`

Estas variables representan **atributos cualitativos discretos**, donde las categorías no pueden interpretarse como valores ordenados (por ejemplo, tipo de contrato, estado civil o tipo de vivienda).

#### Justificación del One-Hot Encoding

- Evita introducir **relaciones ordinales artificiales** entre categorías.
- Permite que los modelos basados en distancia o separación lineal (como regresión logística, K-Means o árboles) interpreten correctamente cada categoría.
- Mantiene la interpretabilidad de los coeficientes y reglas del modelo.

#### Parámetros utilizados

- `drop="first"`:
  - Elimina una categoría de referencia por variable.
  - Reduce dimensionalidad.
  - Previene multicolinealidad perfecta (dummy variable trap).
- `sparse_output=False`:
  - Se genera una matriz densa para facilitar inspección y manipulación directa en el laboratorio.

#### Proceso aplicado

1. Se ajustó el codificador sobre las columnas categóricas seleccionadas.
2. Se generaron nuevas variables dummy con nombres explícitos (`<variable>_<categoría>`).
3. Se eliminaron las columnas categóricas originales.
4. Se incorporaron las variables codificadas al `data_set_base`.

#### Resultado

El dataset base queda compuesto exclusivamente por variables numéricas, completamente preparado para:

- Análisis de correlación
- Escalado
- Modelado supervisado y no supervisado

Además, el número de columnas aumenta de forma controlada y justificada, incorporando información categórica sin introducir ruido estructural ni redundancia semántica.


## Verificación de tipos

In [49]:
remaining_objects = data_set_base.select_dtypes(include=["object"]).columns.tolist()

print("Columnas categóricas restantes:")
print(remaining_objects)

Columnas categóricas restantes:
[]
